# OpenBB Data Base

In [1]:
from openbb import obb
import pandas as pd

ratios = [
    "gross_margin", "operating_margin", "net_margin", "roe", "roce",  # Profitability
    "asset_turnover", "inventory_turnover", "receivables_turnover", "operating_cycle",  # Eficciency
    "current_ratio", "quick_ratio", "debt_to_equity", "interest_coverage",  # Liquidity and Solvency
    "price_to_fcf"  # Valuation
]


In [2]:
all_companies = obb.equity.search("", provider="sec")
len(all_companies.results)

9708

In [3]:
sec_df = all_companies.to_df()

In [4]:
sec_df.head()

,symbol,name,cik
0,AAPL,Apple Inc.,320193
1,MSFT,MICROSOFT CORP,789019
2,NVDA,NVIDIA CORP,1045810
3,AMZN,AMAZON COM INC,1018724
4,GOOGL,Alphabet Inc.,1652044


In [45]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"


tables = pd.read_html(url)
sp500_table = tables[0]
sp500_tickers = sp500_table['Symbol'].tolist()

len(sp500_tickers)

503

In [49]:
stocks = sec_df['symbol'].tolist()
stocks = stocks[350:600]

In [39]:
from dotenv import load_dotenv
import os

key = os.getenv("PAT_OBB")
obb.account.login(pat=key)

In [50]:
from pandas.errors import EmptyDataError

ratio_list = []

for stock in stocks:
    try:
        df = obb.equity.fundamental.ratios(stock, provider="fmp", limit=10).to_df() # limit=10 for 10 year historical data, however, the free plan only allows 5 years
        df['Stock'] = stock
        ratio_list.append(df)

    except EmptyDataError:
        print(f"No data found for the symbol {stock}")

    except Exception as e:
        print(f"Error with {stock}: {e}")


if ratio_list:
    ratio_df = pd.concat(ratio_list, ignore_index=True)
else:
    ratio_df = pd.DataFrame()


Error with BRK.B: 
[Empty] -> No data found for the symbol BRK.B.
Error with BF.B: 
[Empty] -> No data found for the symbol BF.B.
Error with PODD: 
[Error] -> Unauthorized FMP request -> Limit Reach . Please upgrade your plan or visit our documentation for more details at https://site.financialmodelingprep.com/


In [51]:
ratio_df = pd.concat(ratio_list, ignore_index=True)

ratio_df

,period_ending,fiscal_period,fiscal_year,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_inventory_outstanding,operating_cycle,days_of_payables_outstanding,...,price_to_free_cash_flows_ratio,price_to_operating_cash_flows_ratio,price_cash_flow_ratio,price_earnings_to_growth_ratio,price_sales_ratio,dividend_yield,enterprise_value_multiple,price_fair_value,interest_coverage,Stock
0,2024-12-31,FY,2024,1.411158,1.082623,0.497512,48.597355,93.429086,142.026441,67.204264,...,111.446351,39.088935,39.088935,-0.106482,2.893297,0.027875,16.416792,18.506708,NaN,MMM
1,2023-12-31,FY,2023,1.070733,0.755508,0.387854,53.050702,95.255182,148.305884,64.102668,...,9.995352,7.578811,7.578811,0.032353,1.549110,0.065401,-9.122197,10.531820,-9.690021,MMM
2,2022-12-31,FY,2022,1.542371,0.978263,0.383808,48.326857,101.954035,150.280892,60.409474,...,14.770213,10.149734,10.149734,-50.244370,1.657868,0.059369,7.945437,3.842056,14.153680,MMM
3,2021-12-31,FY,2021,1.704815,1.153071,0.505147,48.109178,96.808992,144.918170,58.143655,...,14.696170,11.535724,11.535724,1.711831,2.432111,0.039773,10.343493,5.688119,11.176230,MMM
4,2020-12-31,FY,2020,1.885003,1.351661,0.583040,53.359589,93.178862,146.538450,56.294188,...,12.766253,10.404347,10.404347,0.812508,2.622746,0.040137,10.673665,6.527760,8.453686,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,2024-12-31,FY,2024,2.289021,1.709000,0.847325,67.369869,94.729397,162.099266,75.747601,...,29.249410,26.126988,26.126988,5.221784,5.043755,0.000885,19.121151,3.584985,6.098030,IR
1230,2023-12-31,FY,2023,2.216604,1.668746,0.873146,65.514318,91.489897,157.004215,73.221162,...,24.612604,22.729223,22.729223,1.435874,4.553051,0.001035,19.720039,3.199905,7.430121,IR
1231,2022-12-31,FY,2022,2.369952,1.757407,0.963560,69.220628,104.233436,173.454064,79.156014,...,27.656948,24.615745,24.615745,1.108984,3.579420,0.001530,17.006954,2.302891,7.919574,IR
1232,2021-12-31,FY,2021,2.803638,2.221639,1.437351,67.199557,98.543886,165.743443,77.351528,...,46.542757,41.695656,41.695656,-0.031519,4.980917,0.000320,25.844592,2.851044,6.450399,IR


In [52]:
ratio_df['Stock'].unique()

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL',
       'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT',
       'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP',
       'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS',
       'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET',
       'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY',
       'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BBY', 'TECH', 'BIIB',
       'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR',
       'BRO', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB',
       'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW',
       'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX',
       'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG',
       'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP',
       'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA

In [53]:
ratio_df.columns

Index(['period_ending', 'fiscal_period', 'fiscal_year', 'current_ratio',
       'quick_ratio', 'cash_ratio', 'days_of_sales_outstanding',
       'days_of_inventory_outstanding', 'operating_cycle',
       'days_of_payables_outstanding', 'cash_conversion_cycle',
       'gross_profit_margin', 'operating_profit_margin',
       'pretax_profit_margin', 'net_profit_margin', 'effective_tax_rate',
       'return_on_assets', 'return_on_equity', 'return_on_capital_employed',
       'net_income_per_ebt', 'ebt_per_ebit', 'ebit_per_revenue', 'debt_ratio',
       'debt_equity_ratio', 'long_term_debt_to_capitalization',
       'total_debt_to_capitalization', 'cash_flow_to_debt_ratio',
       'company_equity_multiplier', 'receivables_turnover',
       'payables_turnover', 'inventory_turnover', 'fixed_asset_turnover',
       'asset_turnover', 'operating_cash_flow_per_share',
       'free_cash_flow_per_share', 'cash_per_share', 'payout_ratio',
       'operating_cash_flow_sales_ratio',
       'free_cash_f

In [54]:
df = pd.read_csv("data/stocks_ratios.csv")

ratio_df = ratio_df[df.columns]

In [55]:
ratio_df = ratio_df[~ratio_df['Stock'].isin(df['Stock'])]

ratio_df.head()

,period_ending,fiscal_period,fiscal_year,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_inventory_outstanding,operating_cycle,days_of_payables_outstanding,...,price_to_free_cash_flows_ratio,price_to_operating_cash_flows_ratio,price_cash_flow_ratio,price_earnings_to_growth_ratio,price_sales_ratio,dividend_yield,enterprise_value_multiple,price_fair_value,interest_coverage,Stock
5,2024-12-31,FY,2024,1.552497,0.959429,0.267053,51.756371,82.225445,133.981816,90.971846,...,21.162273,17.233903,17.233903,-8.714159,2.626093,0.018989,12.719370,5.323432,106.985075,AOS
6,2023-12-31,FY,2023,1.587115,1.060933,0.359568,56.462832,76.668497,133.131329,92.544764,...,20.682782,18.442636,18.442636,0.154152,3.208601,0.014844,14.781985,6.702504,63.116667,AOS
7,2022-12-31,FY,2022,1.748769,1.195997,0.418754,56.511362,77.748628,134.259990,94.219775,...,27.592555,22.636611,22.636611,-0.749344,2.360204,0.020000,28.502159,5.069502,24.797872,AOS
8,2021-12-31,FY,2021,1.566500,1.166339,0.396228,65.431631,73.344031,138.775662,122.196364,...,24.254420,21.413199,21.413199,0.662768,3.879172,0.012391,19.078414,7.492633,146.488372,AOS
9,2020-12-31,FY,2020,1.825567,1.486968,0.646621,73.748834,61.292877,135.041711,121.564546,...,17.524455,15.753615,15.753615,-5.751070,3.058442,0.017922,15.868989,4.790947,62.863014,AOS


In [56]:
print(len(df['Stock'].unique()) + len(ratio_df['Stock'].unique()))

867


In [57]:
updated_df = pd.concat([df, ratio_df], ignore_index=True)

In [58]:
len(updated_df['Stock'].unique())

867

In [59]:
updated_df.shape

(4328, 58)

In [60]:
updated_df.to_csv("data/stocks_ratios.csv", index=False)

In [89]:
ratio_df = pd.read_csv("data/stocks_ratios.csv")

In [90]:
ratio_df['period_ending'] = pd.to_datetime(ratio_df['period_ending'])

In [91]:
len(ratio_df['Stock'].unique())

867

In [68]:
ids = ratio_df['Stock'].unique().tolist()

In [70]:
from pandas.errors import EmptyDataError

df_list = []

for id in ids:
    try:
        price = obb.equity.price.historical(symbol=id, start_date="2020-01-01", provider="yfinance").to_df()
        price['Stock'] = id
        df_list.append(price)
    
    except EmptyDataError:
        print(f"No data found for the symbol {id}")

    except Exception as e:
        print(f"Error with {id}: {e}")


if df_list:
    price_df = pd.concat(df_list, ignore_index=False)
else:
    price_df = pd.DataFrame()

In [77]:
price = price_df.copy()

In [16]:
price_df.to_csv("data/stock_prices.csv", index=True)

In [78]:
price_df = pd.read_csv("data/stock_prices.csv")

In [79]:
price_df.head()

,date,open,high,low,close,volume,split_ratio,dividend,Stock
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400,0.0,0.0,AAPL
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800,0.0,0.0,AAPL
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200,0.0,0.0,AAPL
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000,0.0,0.0,AAPL
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200,0.0,0.0,AAPL


In [80]:
price_df['date'] = pd.to_datetime(price_df['date'])

In [81]:
price_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'split_ratio',
       'dividend', 'Stock'],
      dtype='object')

In [82]:
price_df = price_df[price_df['close'].notna()]
len(price_df['Stock'].unique())

841

In [92]:
def find_closest_price(stock, date): #Find the closest date for the period ending date
    stock_prices = price_df[price_df['Stock'] == stock]  # Filter stock prices
    if stock_prices.empty:
        return (pd.NaT, None, None)  # Return None if no prices found
    closest_row = stock_prices.iloc[(stock_prices['date'] - date).abs().argsort()[:1]]
    return tuple(closest_row[['date', 'close', 'split_ratio']].values[0]) if not closest_row.empty else (pd.NaT, None, None)

matched_prices = ratio_df.apply(lambda row: find_closest_price(row['Stock'], row['period_ending']), axis=1)

ratio_df[['closest_date', 'close_price', 'split_ratio']] = pd.DataFrame(matched_prices.tolist(), index=ratio_df.index)

In [93]:
filtered_df = ratio_df[ratio_df['close_price'].notna()]
len(filtered_df['Stock'].unique())

841

In [94]:
df_2 = ratio_df.copy()

In [105]:
price_df['split_ratio'].fillna(1, inplace=True)

/var/folders/n8/s5gn3hhs4z90dmrnk47r93fh0000gn/T/ipykernel_2057/4039875581.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  price_df['split_ratio'].fillna(1, inplace=True)


In [107]:
import numpy as np

def adjust_price(stock, date, price):
    if pd.isna(price) or pd.isna(date):  # Si falta el precio o la fecha, devolver NaN
        return np.nan
    
    stock_splits = price_df[(price_df['Stock'] == stock) & (price_df['date'] > date)]
    
    if stock_splits.empty:
        return price  # Si no hay splits, devolver el precio original
    
    split_factor = stock_splits['split_ratio'].replace(0, 1).cumprod().iloc[-1] if not stock_splits.empty else 1
    
    return price / split_factor if split_factor != 0 else price

ratio_df['adjusted_price'] = ratio_df.apply(lambda row: adjust_price(row['Stock'], row['closest_date'], row['close_price']), axis=1)

In [108]:
filtered_df = ratio_df[ratio_df['adjusted_price'].notna()]
len(filtered_df['Stock'].unique())

841

In [109]:
df_r = ratio_df.copy()

In [110]:
ratio_df = ratio_df.sort_values(by=['Stock', 'period_ending'])

In [111]:
ratio_df['prev_adjusted_price'] = ratio_df.groupby('Stock')['adjusted_price'].shift(1)
ratio_df['return'] = (ratio_df['adjusted_price'] - ratio_df['prev_adjusted_price']) / ratio_df['prev_adjusted_price']

In [114]:
new_column_order = ['Stock', 'return', 'adjusted_price', 'close_price'] + [col for col in ratio_df.columns if col not in ['Stock', 'return', 'adjusted_price', 'close_price']]

ratio_df = ratio_df[new_column_order]

In [116]:
ratio_df.drop(columns=['prev_adjusted_price', 'closest_date','split_ratio'], inplace=True)

In [117]:
ratio_df.head()

,Stock,return,adjusted_price,close_price,period_ending,fiscal_period,fiscal_year,current_ratio,quick_ratio,cash_ratio,...,price_earnings_ratio,price_to_free_cash_flows_ratio,price_to_operating_cash_flows_ratio,price_cash_flow_ratio,price_earnings_to_growth_ratio,price_sales_ratio,dividend_yield,enterprise_value_multiple,price_fair_value,interest_coverage
1846,A,NaN,102.089996,102.089996,2020-10-31,FY,2020,2.327880,1.837082,0.982277,...,43.874562,39.333928,34.251694,34.251694,-1.385299,5.908562,0.007037,23.993947,6.473591,10.846154
1845,A,0.538447,157.059998,157.059998,2021-10-31,FY,2021,2.224239,1.738290,0.868852,...,39.459702,36.841235,32.152350,32.152350,0.557219,7.555980,0.004943,28.400719,8.859944,16.629630
1844,A,-0.119126,138.350006,138.350006,2022-10-31,FY,2022,2.030091,1.472327,0.565825,...,32.987759,40.515818,31.529459,31.529459,6.251966,6.040691,0.006044,21.606143,7.797672,19.261905
1843,A,-0.252837,103.370003,103.370003,2023-10-31,FY,2023,2.611354,1.968185,0.991890,...,24.508694,20.617897,17.150553,17.150553,34.230475,4.447648,0.008720,18.517780,5.199449,14.210526
1842,A,0.260617,130.309998,130.309998,2024-10-31,FY,2024,2.089182,1.576253,0.701319,...,29.317223,27.523598,21.581896,21.581896,5.623576,5.804900,0.007251,26.781519,6.407240,15.500000


In [119]:
stocks = ratio_df[ratio_df['return'].notna()]

In [120]:
len(stocks['Stock'].unique())

841

In [121]:
stocks.shape

(3357, 61)

In [122]:
stocks.to_csv("data/stocks.csv", index=False)